# Отработка ошибок свзязанных с запуском кода

- Если после установки необходимых библиотек возникает ошибка то скорее всего не установлен вебдрайвер Chrome для вашей ОС. Как установить вебдрайвер можно ознакомиться по этой ссылке. [selenium-python](https://selenium-python.com/install-chromedriver-chrome?ysclid=ly2ufnhjip111135754)
- Если после установки вебдрайвера код не отрабатывает либо возникает ошибка то требуется проверить не открыт ли в памяти браузер Chrome. Если да то требуется его выгрузить (закрыть). Это связано с тем что данный код отрабатывает в фоновом режиме но при этом использует вебдрайвер Chrome который божет конфликтовать с самим запущенным браузером Chrome.
- Если ошибка попрежнему возникает то требуется проверить или переустановить используемые библиотеки.

In [1]:
#!pip3 install --upgrade pip
#!pip3 install selenium
#!pip3 install beautifulsoup4
#!pip3 install pandas
#!pip3 install regex

In [2]:
import re
import time
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
class GoldenPagesParser:

    # инициализатор 
    def __init__(self, task_array=[], url_task_array=[]):
        # инициализируем массив запросов
        self.task_array = task_array
        self.url_task_array = url_task_array
        # инициализируем вебдрайвер
        options = Options()
        # гиперпараметр для запуска вебдрайвера селениум без графического интерфейса
        options.add_argument('--headless')
        self.driver = Chrome(options=options)
         # параметр максимального времени загрузки страницы сек
        self.upload_wait_time = 10
        # параметр ожидания после прокликивания для загрузки данных для парсинга сек
        self.sleep_time = 1
        self.df = pd.DataFrame({
            'id':pd.Series(dtype='int'),
            'company_name':[],
            'company_second_name':[],
            'company_third_name':[],
            'address':[],
            'landmarks':[],
            'phone_array':[],
            'site_url':[],
            'parsing_url':[],
            'parsing_task':[],
        })

    # функция проверки элементов на присутствие в блоке, в случае отсутствия добавляет пустое значение в случае присутствия очищает текст от лишних пробелов и табуляции
    def cleaner(self, text_block):
        text = re.sub(r'\s+', ' ', text_block.text.replace('\n', '').strip()).replace('\t ', '\t').replace(' \t', '\t') if text_block else None
        return text

    # функция парсинга ячейки
    def cell_parser(self, div, parsing_url, parsing_task):
        line = {}            
        # забираем данные номера ячейки убираем пробелы табуляцию и лишнюю пунктуацию оставляя только цифры и преобразуем в числовой тип данных
        line['id'] = int(re.sub('[^0-9]', '', div.find('span', attrs={'class':'h3 mb-0 fz-18 clr-blue'}).text)) if div.find('span', attrs={'class':'h3 mb-0 fz-18 clr-blue'}) else None
        line['company_name'] = self.cleaner(div.find('div', attrs={'class':'h3 mb-0'}))
        line['company_second_name'] = self.cleaner(div.find('p', attrs={'class':'d-block mb-0 mx-3 clr-blue'}))
        line['company_third_name'] = self.cleaner(div.find('div', attrs={'class':'gp_name_legal'}))
        line['address'] = self.cleaner(div.find('div', attrs={'class':'gp_wrap_address'}).find('p'))
        line['landmarks'] = self.cleaner(div.find('div', attrs={'class':'gp_wrap_address'}).find('p', attrs={'class':'clr-grey'}))
        # забираем данные блока номеров из ячейки в массив
        phones = div.find_all('a', attrs={'class':'d-flex flex-column'}) if div.find('a', attrs={'class':'d-flex flex-column'}) else []
        # убираем пробелы табуляцию и лишнюю пунктуацию из массива оставляя только цифры и преобразуем в числовой тип данных
        line['phone_array'] = [int(re.sub('[^0-9]', '', phone.get_text(strip=True))) for phone in phones]
        site = div.find('ul', attrs={'class':'gp_listCardIconBottom list-unstyled mb-0 d-flex'}).find_all('li', attrs={'class':'gp_icoActive d-flex d-flex align-items-center justify-content-center'})[-1].get('title') \
            if div.find('ul', attrs={'class':'gp_listCardIconBottom list-unstyled mb-0 d-flex'}).find_all('li', attrs={'class':'gp_icoActive d-flex d-flex align-items-center justify-content-center'}) else []
        # забираем только адрес сайта если он присутствует и массив не пуст чистим адреса сайта для сохранения убираем протокол, хост и путь оставляя только адрес айта
        line['site_url'] = site.replace('https://', '').replace('http://', '').replace('www.', '').split('\\')[0] if site and site not in ['Instagram', 'Telegram', 'Facebook', None] else None
        line['parsing_url'] = parsing_url
        line['parsing_task'] = parsing_task
        return line

    # функция добавления данных со всей страницы в итоговый фрейм
    def page_parser(self, divs_content, url, parsing_task):
        for div in divs_content:
            self.df.loc[len(self.df)] = self.cell_parser(div, url, parsing_task)

    #  функция для пркликивания элементов страницы и возврата блока парсинга страницы
    def clicker(self, parsing_url):
        self.driver.get(parsing_url)
        # ожидание загрузки данных
        wait = WebDriverWait(self.driver, self.upload_wait_time)
        # задаем параметры искомого элемента на странице и осуществляем поиск всех элементов соответствующих параметру для дальнейшего взаимодействия если элементов нет то выводим сообщение об отсутствии
        try:
            buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//button[contains(text(), 'Показать номер')]")))
            if buttons:
                for button in buttons:
                    try:
                        self.driver.execute_script('arguments[0].click();', button)
                    except Exception as e:
                        print('Ошибка при нажатии на элемент:', e)
                # время ожидания для загрузки элемента
                time.sleep(self.sleep_time)
            else:
                print('Элементы не найдены на странице.')
        except TimeoutException:
            pass
        # передаем содержимое вебдрайвера после взаимодействия со страницей из селениума в бьютифул суп для парсинга страницы
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        # ищем блок содержимого тела страницы
        if soup.find('div', class_='tab-pane fade'):
            divs = soup.find('div', class_='tab-pane fade')
        elif soup.find('div', class_='col-xxl-10 col-9'):
            divs =  soup.find('div', class_='col-xxl-10 col-9')
            #divs = soup.find('div', class_='col-xxl-10 col-9').find('div', class_='d-xxl-flex gap-20').find('div', class_= False)
        else:
            divs = []
        return divs

    # функция генерации адреса страниц и парсинга всего запроса
    def request_parser(self, parsing_task):
        # генерация изменяемой части ссылки поиска зпроса
        start_task_url_request_part = parsing_task.replace(' ', '%20')
        # генерация изменяемой части альтернативной ссылки поиска запроса для страниц 2 и последующих если запрос не выдал ссылку
        second_page_task_url_request_part = parsing_task.replace(' ', '+')
        # сбор ссылки
        parsing_url = 'https://www.goldenpages.uz/search/?region=&s=' + str(start_task_url_request_part)
        # массив сылок результата запроса
        new_url_array = []
        # номер обрабатываемой страницы
        page_number = 1
        # блок парсинга страницы представлен в виде масссива искомых элементов
        divs = self.clicker(parsing_url)
        # ищем все ячейки страницы с требуемыми данными если их нет возвращаем пустой массив, отрабатываются ошибки 404 или заход на страницу за диапазон данных где 404 не появляется
        divs_content = divs.find_all('section', class_=re.compile(r'gp_company gp_company_')) if divs.find('section', class_=re.compile(r'gp_company gp_company_')) else []
        #цикл парсинга страницы и сбора URL новой сылки тогоже запроса
        while len(divs_content) > 0:
            self.page_parser(divs_content, parsing_url, parsing_task)
            print(f'Page N:{page_number}, Task: {parsing_task}')
            # защита от бесконечного цикла на случай если что-то пойдет не так, цикл не может превышать 100 итераций на один запрос поиска
            if page_number == 100:
                break
            page_number += 1
            # сборка адреса обрабатывемой страницы
            parsing_url = 'https://www.goldenpages.uz/search/?region=&s=' + str(second_page_task_url_request_part) + '&tab=tab1&Page1=' + str(page_number)
            divs = self.clicker(parsing_url)
            divs_content = divs.find_all('section', class_=re.compile(r'gp_company gp_company_')) if divs.find('section', class_=re.compile(r'gp_company gp_company_')) else []
        # ищем элемент новых сылок в случае если запрос выдал сылки в место результата для парсинга
        divs_new_task_array = divs.find('ul', class_='gp_dop_rubrics list-unstyled ddd').find_all('li') if divs.find('ul', class_='gp_dop_rubrics list-unstyled ddd') else []
        # если элемент содержит данные то запускаем сбор ссылок
        if len(divs_new_task_array) > 0:
            # перебор масива данных так как сылка может быть не одна
            for div_new_task in divs_new_task_array:
                task = {}
                new_url_task = div_new_task.find('a', href=True).get('href')
                # собираем вторичную сылку
                task['new_url'] = 'https://www.goldenpages.uz' + str(new_url_task)
                # номер сылки в ирархии выданных сылок
                list_number = int(re.sub('[^0-9]', '', div_new_task.find('span', class_='gp_list_number').text))
                list_name = self.cleaner(div_new_task.find('a', attrs={'target':'_blank'})) if div_new_task.find('a', attrs={'target':'_blank'}) else ''
                list_data_count = int(re.sub('[^0-9]', '', div_new_task.find('span', class_='gp_badge').text)) if div_new_task.find('span', class_='gp_badge') else 0
                task['second_description'] = (f'Second task( №: {list_number}, Name: {list_name}, Data count: {list_data_count})')
                # пополняем архив сылок
                new_url_array.append(task)
            #Запускаем цикл обхода и парсинга по найденным сылкам страницы 
            for new_parsing_url in new_url_array:
                page_number = 1
                new_parsing_url_second_page = new_parsing_url['new_url']
                # прокликиваем первую страницу сылок
                divs_second = self.clicker(new_parsing_url['new_url'])
                # цикл обхода по всем страницам сылок
                while True:
                    divs_second_content = divs_second.find_all('section', class_=re.compile(r'gp_company gp_company_')) if divs_second.find('section', class_=re.compile(r'gp_company gp_company_')) else []
                    self.page_parser(divs_second_content, new_parsing_url_second_page, parsing_task + ' ' + new_parsing_url['second_description'])
                    print(f'Page N:{page_number}, Task: {parsing_task}, {new_parsing_url["second_description"]}')
                    page_number += 1
                    # собираем новую сылку вторых и последующих страниц
                    new_parsing_url_second_page = new_parsing_url['new_url'] + '&Page=' + str(page_number)
                    # передаем новую сылку в драйвер и заодно прокликиваем ее в случае если есть результат
                    divs_second = self.clicker(new_parsing_url_second_page)
                    # проверяем не рава ли сылка возвращенная драйвером стартовой сылке, если да то результаты обхода вышли за пределы результата данных категории и опять попали на стартовую сылку
                    if self.driver.current_url == new_parsing_url['new_url'] or page_number == 101:
                        break
                
    # функция парсинга данных всего массива запросов
    def task_frame_parser(self):
        driver = self.driver
        for task in self.task_array:
            parsing_task = re.sub(r'\s+', ' ', task.replace('\n', '').strip()).replace('\t ', '\t').replace(' \t', '\t') if task else ''
            self.request_parser(parsing_task) if parsing_task != '' else print('Введен пустой запрос.')
        print('Список запросов пуст.') if len(self.task_array) <= 0 else print('Сбор данных завершен.')
        # закрываем браузер в селениум
        driver.quit()
        return self.df
    
    # функция парсинга массива URL результатов запросов
    def task_url_parser(self):
        # вызываем вебдрайвер
        driver = self.driver
        # счетчик номера таска
        task_number = 0
        # итератор запросов
        for parsing_url in self.url_task_array:
            # массив URL сылок содержащихся на странице в случае если сылка содержит сылки рубрик для парсинга а не сами данные
            new_url_array = []
            # учеличение на 1 номера запроса с каждой итерацией
            task_number += 1
            # проверка содержания сылки
            if parsing_url != '':
                # установка номера страницы на новом запросе
                page_number = 1
                # проверка кейса парсинга
                if 'goldenpages.uz/search/' in parsing_url:
                    # прокликивание страницы
                    divs = self.clicker(parsing_url)
                    # сбор информации о запросе
                    parsing_task = re.findall(r'"(.*?)"', BeautifulSoup(self.driver.page_source, 'html.parser').find('h1', class_='h1 clr-blue px-2 mb-3').text)[0] \
                        if BeautifulSoup(self.driver.page_source, 'html.parser').find('h1', class_='h1 clr-blue px-2 mb-3') else f'task №: {task_number}'
                    # доступ к данным первой страницы
                    try:
                        divs_content = divs.find_all('section', class_=re.compile(r'gp_company gp_company_'))
                    except:
                        divs_content = []
                    # итератор вторых и последующих страниц запроса
                    while len(divs_content) > 0:
                        # сбор данных страницы
                        self.page_parser(divs_content, parsing_url, parsing_task)
                        # вывод результата о успешном парсинге
                        print(f'Page N:{page_number}, Task: {parsing_task}')
                        # защита от бесконечного цикла на случай если что-то пойдет не так, цикл не может превышать 100 итераций на один запрос поиска
                        if page_number == 100:
                            break
                        # увеличение номера страницы с каждой итерацией
                        page_number += 1
                        # сборка адреса обрабатывемой страницы
                        parsing_url = parsing_url.replace('%20', '+') + '&tab=tab1&Page1=' + str(page_number)
                        # прокликивание новой сылки
                        divs = self.clicker(parsing_url)
                        # доступ к данным новой страницы
                        divs_content = divs.find_all('section', class_=re.compile(r'gp_company gp_company_')) if divs.find('section', class_=re.compile(r'gp_company gp_company_')) else []
                    # ищем элемент новых сылок в случае если запрос выдал сылки в место результата для парсинга
                    try:
                        divs_new_task_array = divs.find('ul', class_='gp_dop_rubrics list-unstyled ddd').find_all('li')
                    except:
                        divs_new_task_array = []
                    # если элемент содержит данные то запускаем сбор ссылок
                    if len(divs_new_task_array) > 0:
                        # перебор масива данных так как сылка может быть не одна
                        for div_new_task in divs_new_task_array:
                            # объявляем пустой вторичный таск
                            task = {}
                            # получаем часть вторичной сылку
                            new_url_task = div_new_task.find('a', href=True).get('href')
                            # собираем вторичную сылку
                            task['new_url'] = 'https://www.goldenpages.uz' + str(new_url_task)
                            # номер сылки в ирархии выданных сылок
                            list_number = int(re.sub('[^0-9]', '', div_new_task.find('span', class_='gp_list_number').text))
                            # наиминование группы вторичной сылки
                            list_name = self.cleaner(div_new_task.find('a', attrs={'target':'_blank'})) if div_new_task.find('a', attrs={'target':'_blank'}) else ''
                            # число данных внутри сылки
                            list_data_count = int(re.sub('[^0-9]', '', div_new_task.find('span', class_='gp_badge').text)) if div_new_task.find('span', class_='gp_badge') else 0
                            # группируем наименование вторичного таска
                            task['second_description'] = (f'Second task( №: {list_number}, Name: {list_name}, Data count: {list_data_count})')
                            # пополняем архив сылок
                            new_url_array.append(task)
                        #Запускаем цикл обхода и парсинга по найденным сылкам страницы 
                        for new_parsing_url in new_url_array:
                            # устанавливаем номер страницы вторичной сылки
                            page_number = 1
                            # копируем вторичную сылку для проверки на выход за диапазон запросов
                            new_parsing_url_second_page = new_parsing_url['new_url']
                            # прокликиваем первую страницу сылок
                            divs_second = self.clicker(new_parsing_url['new_url'])
                            # цикл обхода по всем страницам сылок
                            while True:
                                # получаем доступ к первой страницы вторичного запроса
                                divs_second_content = divs_second.find_all('section', class_=re.compile(r'gp_company gp_company_')) if divs_second.find('section', class_=re.compile(r'gp_company gp_company_')) else []
                                # забираем данные страницы
                                self.page_parser(divs_second_content, new_parsing_url_second_page, parsing_task + ' ' + new_parsing_url['second_description'])
                                # выводим результат
                                print(f'Page N:{page_number}, Task: {parsing_task}, {new_parsing_url["second_description"]}')
                                # увеличиваем счетчик итератора страниц
                                page_number += 1
                                # собираем новую сылку вторых и последующих страниц
                                new_parsing_url_second_page = new_parsing_url['new_url'] + '&Page=' + str(page_number)
                                # передаем новую сылку в драйвер и заодно прокликиваем ее в случае если есть результат
                                divs_second = self.clicker(new_parsing_url_second_page)
                                # проверяем не рава ли сылка возвращенная драйвером стартовой сылке, если да то результаты обхода вышли за пределы результата данных категории и опять попали на стартовую сылку
                                if self.driver.current_url == new_parsing_url['new_url'] or page_number == 101:
                                    break
                # проверяем сылку запроса на соответствие ворому кейсу сылок
                elif 'goldenpages.uz/rubrics/' in parsing_url:
                    # прокликиваем страницу
                    divs = self.clicker(parsing_url)
                    # забираем наименование запроса со страницы
                    parsing_task = BeautifulSoup(self.driver.page_source, 'html.parser').find('h1', class_='h1 clr-blue mb-1 mt-0').text.strip() \
                        if BeautifulSoup(self.driver.page_source, 'html.parser').find('h1', class_='h1 clr-blue mb-1 mt-0') else f'task №: {task_number}'
                    # копируем стартовую сылку запроса для выхода из цикла по превышению диапазона
                    parsing_url_second_page = parsing_url
                    # итератор страниц запроса
                    while True:
                        # пытаемся получить данные со страницы
                        try:
                            divs_content = divs.find_all('section', class_=re.compile(r'gp_company gp_company_'))
                        except:
                            divs_content = []
                        # запускаем парсер
                        self.page_parser(divs_content, parsing_url_second_page, parsing_task)
                        # выводим результат
                        print(f'Page N:{page_number}, Task: {parsing_task}')
                        # увеличиваем счетчик страниц
                        page_number += 1
                        # собираем новую сылку вторых и последующих страниц
                        parsing_url_second_page = parsing_url + '&Page=' + str(page_number)
                        # передаем новую сылку в драйвер и заодно прокликиваем ее в случае если есть результат
                        divs = self.clicker(parsing_url_second_page)
                        # проверяем не рава ли сылка возвращенная драйвером стартовой сылке, если да то результаты обхода вышли за пределы результата данных категории и опять попали на стартовую сылку
                        if self.driver.current_url == parsing_url or page_number == 101:
                            break
                else:
                    print('Введен не корректный адрес')
                    break
            else:
                print('Введен пустой запрос.')
        print('Список запросов пуст.') if len(self.url_task_array) <= 0 else print('Сбор данных завершен.')
        # закрываем вебдрайвер
        driver.quit()
        return self.df

# Работа с парсером
- Для запуска парсинга сайта [goldenpages.uz](https://www.goldenpages.uz/search/) требуется составить массив запросов поиска сайта, переменная task_array = ['кафе в самарканде', '', 'кафе в намангане', 'кафе в бухаре']. Ограничение сайта GoldenPages - запросы должны содержать минимум три символа, иначе вывод резултата поиска сайта будет пустым.
- Для старта парсинга по массиву запросов и сохранения результипующего фрейма спарсенных данных требуется вызвать команду parsed_data = GoldenPagesParser(task_array = task_array).task_frame_parser() которая вернет Pandas Data Frame с результатом парсинга в переменную parsed_data.
- Для запуска парсинга сайта [goldenpages.uz](https://www.goldenpages.uz/search/) требуется составить массив запросов поиска сайта, переменная url_task_array = ['https://www.goldenpages.uz/search/?region=&s=кафе%20в%20самарканде', '', 'https://www.goldenpages.uz/search/?region==кафе%20в%20самарканде', 'https://www.goldenpages.uz/rubrics/?Id=3480', 'https://www.goldenpages.uz/rubrics/?Id=105284', 'https://www.goldenpages.uz/search/?region=&s=кафе', 'https://www.goldenpages.uz/rubrics/?Id=3052']. Ограничение - запросы должны содержать только URL первых страниц интересующих результатов поиска остальные итератор соберет самостоятельно, если сылка будет содержать результат в виде сылок найденных групп такой запрос итератор тоже отработает самостоятельно собрав все данные всех страниц которые найдет итератор по этим сылкам дополнительно их включать в запрос не требуется.
- Для старта парсинга по массиву URL запросов и сохранения результипующего фрейма спарсенных данных требуется вызвать команду parsed_data = GoldenPagesParser(url_task_array = url_task_array).task_frame_parser() которая вернет Pandas Data Frame с результатом парсинга в переменную parsed_data.
- Код будет содержать следующие колонки собранных данных: id тип 'int' - номер собранной ячейки каждого запроса, company_name, company_second_name, company_third_name, address, landmarks, phone_array, site_url, parsing_url - сылка на страницу с которой были собраны данные, parsing_task - запрос при выполнении которого были собраны данные.
- Для сохранения результирующего фрейма в файл CSV требуется раскоментить строку кода parsed_data.to_csv(f"Selenium_Soup_GoldenPages_{datetime.now().strftime('%d%m%Y')}.csv") которая сохранить данные в файл с наименованием "Selenium_Soup_GoldenPages_01072024.csv" а папку с запускаемом кодом. Дата будет подставляться автоматически - текущая дата сохранения результата парсинга.

In [4]:
# массив запросов для парсинга, ограничение сайта GoldenPages - запросы должны содержать минимум три сивола иначе вывод резултата поиска будет пустым
#task_array = ['кафе в самарканде', '', 'кафе в намангане', 'кафе в бухаре']
#task_array = ['Такси Алат', 'Такси Ангрен', 'Такси Бекабад', 'Такси Вабкент', 'Такси Каттакурган', 'Такси Гиждуван', 'Такси Джандор', 'Такси Каган', 'Такси Карши', 'Такси Навои', 'Такси Ромитан', 'Такси Ромитанский р-не', 'Такси Самарканд', 'Такси Ташкент', 'Такси Бухарский р-не']
#task_array = ['Такси Ташкент', 'Такси Самарканд', '', 'Такси в Узбекистане, заказ такси в Узбекистане']
#task_array = ['кафе']
#task_array = ['Кафельные работы в Ташкенте, в Узбекистане - услуги']
# создаем экземпляр класса парсинга, вызываем парсинг тасков, и сохраняем фрейм данных одной строкой
#parsed_data = GoldenPagesParser(task_array = task_array).task_frame_parser()

# массив URL запросов для парсинга, ограничение сайта GoldenPages - запросы должны содержать минимум три сивола иначе вывод резултата поиска будет пустым
url_task_array = ['https://www.goldenpages.uz/search/?region=&s=кафе%20в%20самарканде', '', 'https://www.goldenpages.uz/search/?region==кафе%20в%20самарканде', 'https://www.goldenpages.uz/rubrics/?Id=3480', 'https://www.goldenpages.uz/rubrics/?Id=105284', 'https://www.goldenpages.uz/search/?region=&s=кафе', 'https://www.goldenpages.uz/rubrics/?Id=3052']
# создаем экземпляр класса парсинга, вызываем парсинг тасков, и сохраняем фрейм данных одной строкой
parsed_data = GoldenPagesParser(url_task_array = url_task_array).task_url_parser()

# сохраняем фрейм в файл текущей датой
#parsed_data.to_csv(f"Selenium_Soup_GoldenPages_{datetime.now().strftime('%d%m%Y')}.csv")

Page N:1, Task: кафе в самарканде
Page N:2, Task: кафе в самарканде
Page N:3, Task: кафе в самарканде
Введен пустой запрос.
Page N:1, Task: Кафе в Самарканде
Page N:2, Task: Кафе в Самарканде
Page N:3, Task: Кафе в Самарканде
Page N:1, Task: Кафельные работы в Ташкенте, в Узбекистане - услуги
Page N:1, Task: кафе, Second task( №: 1, Name: Кафе в Андижане, Data count: 11)
Page N:1, Task: кафе, Second task( №: 2, Name: Кафе в Бухаре, Data count: 6)
Page N:1, Task: кафе, Second task( №: 3, Name: Кафе в Намангане, Data count: 13)
Page N:1, Task: кафе, Second task( №: 4, Name: Кафе в Самарканде, Data count: 43)
Page N:2, Task: кафе, Second task( №: 4, Name: Кафе в Самарканде, Data count: 43)
Page N:3, Task: кафе, Second task( №: 4, Name: Кафе в Самарканде, Data count: 43)
Page N:1, Task: кафе, Second task( №: 5, Name: Кафе в Сукоке, Data count: 5)
Page N:1, Task: кафе, Second task( №: 6, Name: Кафе в Ташкенте, Data count: 656)
Page N:2, Task: кафе, Second task( №: 6, Name: Кафе в Ташкенте, 

In [5]:
display(parsed_data.info())
display(parsed_data.shape)
display(parsed_data.isnull().sum())
display(parsed_data[-50:])

<class 'pandas.core.frame.DataFrame'>
Index: 2362 entries, 0 to 2361
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   2362 non-null   int64 
 1   company_name         2362 non-null   object
 2   company_second_name  774 non-null    object
 3   company_third_name   1189 non-null   object
 4   address              2362 non-null   object
 5   landmarks            2131 non-null   object
 6   phone_array          2362 non-null   object
 7   site_url             300 non-null    object
 8   parsing_url          2362 non-null   object
 9   parsing_task         2362 non-null   object
dtypes: int64(1), object(9)
memory usage: 203.0+ KB


None

(2362, 10)

id                        0
company_name              0
company_second_name    1588
company_third_name     1173
address                   0
landmarks               231
phone_array               0
site_url               2062
parsing_url               0
parsing_task              0
dtype: int64

,id,company_name,company_second_name,company_third_name,address,landmarks,phone_array,site_url,parsing_url,parsing_task
2312,5,"""SUBWAY""","""SUBWAY TASHKENT"" OOO",None,"Узбекистан, Город: Ташкент, Район: Алмазарский...","ШКОЛА №1, МЕЧЕТЬ ТОХТАБОЙ","[998951950400, 998951954144, 998977378704]",None,https://www.goldenpages.uz/rubrics/?Id=107217,"кафе Second task( №: 49, Name: Сеть кафе (фили..."
2313,1,"""WOK"" КАФЕ","""SOLO ADELANTE"" ООО",WOK - Ц1,"100000, Узбекистан, Город: Ташкент, Район: Мир...","ПАМЯТНИК ШАСТРИ, КАФЕ BIBIGON НА Ц1","[998712339712, 998712007373]",wok.uz,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2314,2,"""WOK"" КАФЕ ФИЛИАЛ","""ASIUS VICTUS"" ООО",None,"100070, Узбекистан, Город: Ташкент, Район: Якк...","ПОСОЛЬСТВО СЛОВАКИИ, ""POWER HIGH SCHOOL"" ЧАСТН...",[998712007373],None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2315,3,"""WOK"" КАФЕ ФИЛИАЛ",None,"WOK - ТРЦ ""ATLAS""","100097, Узбекистан, Город: Ташкент, Район: Чил...","""MEDIA PARK"" - ЧИЛАНЗАР, ТРЦ ""ATLAS"" НА ЧИЛАНЗАРЕ",[998712007373],None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2316,4,"""WOK"" КАФЕ ФИЛИАЛ",None,"WOK - ТРЦ ""SAMARQAND DARVOZA"", КАРАТАШ","100021, Узбекистан, Город: Ташкент, Район: Шай...","ТРЦ ""SAMARQAND DARVOZA"", ПЕРЕКРЕСТОК САМАРКАНД...","[998712007373, 998998193300]",None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2317,5,"""WOK"" КАФЕ ФИЛИАЛ",None,"WOK - ТРЦ ""MEGA PLANET""","100194, Узбекистан, Город: Ташкент, Район: Юну...","БАНКЕТНЫЙ ЗАЛ ""GRAND HALL"", ТРЦ ""MEGA PLANET""",[998712007373],None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2318,6,"""WOK"" КАФЕ ФИЛИАЛ",None,"WOK - ТРЦ ""CHIMGAN MALL"" (бывш. ТРЦ ""АТЛАС"" ЧИ...","100125, Узбекистан, Город: Ташкент, Район: Мир...","ТРЦ ""ATLAS-ЧИМГАН"", ""INTERMEBEL"" МЕБЕЛЬНЫЙ САЛОН",[998712007373],None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2319,7,"""WOK"" КАФЕ ФИЛИАЛ",None,"WOK - CLUN CASTLE, DISNEY","Узбекистан, Город: Ташкент, Район: Мирабадский...","""MAGNIT"" СУПЕРМАРКЕТ, РИСОВЫЙ БАЗАР (ФЕРГАНСКИ...",[998712007373],None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2320,8,"""WOK"" КАФЕ ФИЛИАЛ",None,"WOK - АНХОР ЛОКОМОТИВ, LABZAK","100128, Узбекистан, Город: Ташкент, Район: Шай...","СУПЕРМАРКЕТ ""MAKRO"" - АНХОР ЛОКОМОТИВ, РЕСТОРА...",[998712007373],None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
2321,9,"""WOK"" КАФЕ ФИЛИАЛ",None,WOK - СЕРГЕЛИ,"Узбекистан, Город: Ташкент, Район: Сергелийски...","""KAPITALBANK"" СЕРГЕЛИЙСКИЙ ФИЛИАЛ, СУПЕРМАРКЕТ...",[998712007373],None,https://www.goldenpages.uz/rubrics/?Id=106749,"кафе Second task( №: 50, Name: Сеть кафе (фили..."
